Event: iQuHack 2023
Title: Quantinuum
Team: Jaydin, David, Eimaan, Jack, Ken
Description: This QAOA (Quantum Approximate Optimisation algorithm) implementaion uses the most naive possible classical optimisation strategy. Parameters are sampled from a uniform distribution and if a list of parameters increases the value of the cost function these values are stored as the best guess so far. Can you improve on this using a more sophisticated optimisation strategy? COBAYLA and SPSA are two possible methods.

